In [16]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
# from pykeops.torch import LazyTensor
from sklearn.cluster import KMeans

In [21]:
#Read Data, Drop unnecessary column and change - in rating to -1
def getData(path):
    supermarkets = pd.read_csv(path)
    supermarkets.drop('Unnamed: 0', axis=1, inplace=True)
    supermarkets.loc[supermarkets['rating']=='-', 'rating'] = -1
    supermarkets.loc[supermarkets['rating']=='0.0', 'rating'] = -1
    supermarkets.rating = supermarkets.rating.map(float)
    return supermarkets
def cluster():
    #Try Kmeans to fill out the missing values
    km = KMeans(n_clusters=6)
    x = supermarkets.drop(['rating', 'business_status', 'name', 'latitude', 'longitude', 'bus_stations','train_stations'],axis=1).values
    yp = km.fit_predict(x)
    supermarkets[['rating', 'predicted_rating']].head(30)
def generateRating(supermarkets, coefficient):
    data = supermarkets.drop(['business_status','name','latitude','longitude','bus_stations','train_stations'],axis=1)
    data=(data-data.mean())/data.std()
    data['new_rating'] = 0.4 * data['rating'] + 0.4 * data['atms'] + 0.5 *data['banks'] \
              + 0.4 * data['churches'] + 0.4 * data['gas_stations'] + 0.5 *data['hospitals'] \
              + 0.4 * data['mosques'] + 0.4 * data['pharmacies'] + 0.5 *data['restaurants'] \
              + 0.4 * data['schools'] + 0.4 * data['Males'] + 0.5 *data['Females'] \
              + 0.4 * data['Children'] + 0.4 * data['Working'] + 0.5 *data['Elderly']
    data.drop('rating', axis=1, inplace=True)
    return data
data =getData('supermarkets_cleaned_with_popn_v2.csv')
data = generateRating(data, None)
data.head()



,atms,banks,churches,gas_stations,hospitals,mosques,pharmacies,restaurants,schools,Males,Females,Children,Working,Elderly,new_rating
0,-0.518810,-0.862814,-0.633079,0.080125,-0.723125,-1.040533,-1.092615,-0.950018,-1.287014,0.317576,0.377080,0.305240,0.390954,-0.536509,-3.053376
1,0.813666,0.482228,0.444106,-0.705832,0.962816,-0.033729,0.122986,0.307916,0.520830,0.775200,0.920317,0.630812,0.900079,1.638179,4.154013
2,-0.709163,-0.563915,-0.363783,-0.312854,-0.835521,1.224775,-0.397986,-0.525261,-0.081785,-0.125006,-0.274286,-0.264675,-0.202537,0.772058,-1.521088
3,-0.518810,0.033881,-0.094487,-0.705832,-0.385936,-1.040533,0.817616,1.092083,0.445503,0.317576,0.377080,0.305240,0.390954,-0.536509,-0.057230
4,-0.518810,0.382595,1.386643,1.259060,0.962816,-0.788832,0.122986,0.389600,-0.006458,0.317576,0.377080,0.305240,0.390954,-0.536509,1.460715


In [26]:
# data.to_csv('data_regression.csv',index=False, header=False)
data.shape

(1388, 15)

In [2]:
# pytorch mlp for regression
from numpy import vstack
from numpy import sqrt
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch import Tensor
from torch.nn import Linear
from torch.nn import Sigmoid
from torch.nn import Module
from torch.optim import SGD
from torch.nn import MSELoss
from torch.nn.init import xavier_uniform_

# dataset definition
class CSVDataset(Dataset):
    # load the dataset
    def __init__(self, path):
        # load the csv file as a dataframe
        df = read_csv(path, header=None)
        # store the inputs and outputs
        self.X = df.values[:, :-1].astype('float32')
        self.y = df.values[:, -1].astype('float32')
        # ensure target has the right shape
        self.y = self.y.reshape((len(self.y), 1))

    # number of rows in the dataset
    def __len__(self):
        return len(self.X)

    # get a row at an index
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]

    # get indexes for train and test rows
    def get_splits(self, n_test=0.33):
        # determine sizes
        test_size = round(n_test * len(self.X))
        train_size = len(self.X) - test_size
        # calculate the split
        return random_split(self, [train_size, test_size])

# model definition
class MLP(Module):
    # define model elements
    def __init__(self, n_inputs):
        super(MLP, self).__init__()
        # input to first hidden layer
        self.hidden1 = Linear(n_inputs, 10)
        xavier_uniform_(self.hidden1.weight)
        self.act1 = Sigmoid()
        # second hidden layer
        self.hidden2 = Linear(10, 8)
        xavier_uniform_(self.hidden2.weight)
        self.act2 = Sigmoid()
        # third hidden layer and output
        self.hidden3 = Linear(8, 1)
        xavier_uniform_(self.hidden3.weight)

    # forward propagate input
    def forward(self, X):
        # input to first hidden layer
        X = self.hidden1(X)
        X = self.act1(X)
         # second hidden layer
        X = self.hidden2(X)
        X = self.act2(X)
        # third hidden layer and output
        X = self.hidden3(X)
        return X

# prepare the dataset
def prepare_data(path):
    # load the dataset
    dataset = CSVDataset(path)
    # calculate split
    train, test = dataset.get_splits()
    # prepare data loaders
    train_dl = DataLoader(train, batch_size=32, shuffle=True)
    test_dl = DataLoader(test, batch_size=1024, shuffle=False)
    return train_dl, test_dl

# train the model
def train_model(train_dl, model):
    # define the optimization
    criterion = MSELoss()
    optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)
    # enumerate epochs
    for epoch in range(500):
        # enumerate mini batches
        for i, (inputs, targets) in enumerate(train_dl):
            # clear the gradients
            optimizer.zero_grad()
            # compute the model output
            yhat = model(inputs)
            # calculate loss
            loss = criterion(yhat, targets)
            # credit assignment
            loss.backward()
            # update model weights
            optimizer.step()

# evaluate the model
def evaluate_model(test_dl, model):
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(test_dl):
        # evaluate the model on the test set
        yhat = model(inputs)
        # retrieve numpy array
        yhat = yhat.detach().numpy()
        actual = targets.numpy()
        actual = actual.reshape((len(actual), 1))
        # store
        predictions.append(yhat)
        actuals.append(actual)
    predictions, actuals = vstack(predictions), vstack(actuals)
    # calculate mse
    mse = mean_squared_error(actuals, predictions)
    return mse

# make a class prediction for one row of data
def predict(row, model):
    # convert row to data
    row = Tensor([row])
    # make prediction
    yhat = model(row)
    # retrieve numpy array
    yhat = yhat.detach().numpy()
    return yhat

# prepare the data
path = 'data_regression.csv'
train_dl, test_dl = prepare_data(path)
print(len(train_dl.dataset), len(test_dl.dataset))
# define the network
model = MLP(15)
# train the model
train_model(train_dl, model)
# evaluate the model
mse = evaluate_model(test_dl, model)
print('MSE: %.3f, RMSE: %.3f' % (mse, sqrt(mse)))
# make a single prediction (expect class=1)
# row = [0.00632,18.00,2.310,0,0.5380,6.5750,65.20,4.0900,1,296.0,15.30,396.90,4.98]
# yhat = predict(row, model)
# torch.save(model, 'lin_model.h5')
# print('Predicted: %.3f' % yhat)

368 182
MSE: 0.007, RMSE: 0.085


In [3]:
row = [-0.593369154105506,-1.0217534331271696,-0.5337476389488008,0.8366289313353069,-0.5359795978170312,-0.26459234552837585,0.09537639766319944,-1.0175072314624598,-0.8610067601718433,0.45093392563549883,0.4835249295159094,0.605984487187577,0.41832272900476153,-0.33808441144945045,-0.8495007528106202]
# -2.3514740192009241
yhat = predict(row, model)
yhat

array([[-2.37537]], dtype=float32)

In [7]:
import torch
torch.save(model, 'regression_model.h5')
# model = torch.load('regression_model.h5')


array([[-2.37537]], dtype=float32)